In [ ]:
class CatboostWrapper(object):
    def __init__(self, estimator, seed=0, cat_features=None, params=None, metric=None):
        params['random_seed'] = seed
        params['eval_metric'] = metric
        self.estimator = estimator(**params)
        self.cat_features = cat_features

    def train(self, x_train, y_train, x_val, y_val):
        train_pool = Pool(x_train, y_train, cat_features=self.cat_features)
        val_pool = Pool(x_val, y_val, cat_features=self.cat_features) 
        self.estimator.fit(train_pool, eval_set=val_pool, early_stopping_rounds=30, verbose=50)

    def predict(self, x):
        test_pool = Pool(x, cat_features=self.cat_features)
        return self.estimator.predict(x)
        
        
class LightGBMWrapper(object):
    def __init__(self, estimator, seed=0, cat_features=None, params=None, metric=None):
        params['random_state'] = seed
        self.estimator = estimator(**params, num_threads=-1)
        self.metric = metric
        self.cat_features = cat_features

    def train(self, x_train, y_train, x_val, y_val):
        self.estimator.fit(x_train, y_train, eval_set = [(x_train, y_train), (x_val, y_val)],
                           eval_metric= self.metric, early_stopping_rounds=30, verbose=50, categorical_feature=self.cat_features)

    def predict(self, x):
        return self.estimator.predict(x)
    

class XgbWrapper(object):
    def __init__(self, estimator, seed=0, params=None, metric=None):
        params['random_state'] = seed
        self.estimator = estimator(**params)
        self.metric = metric

    def train(self, x_train, y_train, x_val, y_val):
        self.estimator.fit(x_train, y_train, eval_set = [(x_train, y_train), (x_val, y_val)],
                           eval_metric= self.metric, early_stopping_rounds=30, verbose=50)

    def predict(self, x):
        return self.estimator.predict(x)

In [ ]:
# fix number of iterations, then use default values for the rest parameters of each model

lightgbm_params = {'n_estimators':500}
xgb_params = {'n_estimators':500}
cat_params = {'iterations':500}

In [ ]:
%%time
SEED=123
cat_vars = [var for var in data.columns if data[var].dtypes == 'O']
lg = LightGBMWrapper(LGBMRegressor, SEED, cat_vars, lightgbm_params, 'rmse')
lg.train(train_x, train_y, val_x, val_y)

In [ ]:
%%time
SEED=123
xg = XgbWrapper(XGBRegressor, SEED, xgb_params, 'rmse')
xg.train(train_x, train_y, val_x, val_y)

In [ ]:
%%time
SEED=123
cat_vars = [var for var in data.columns if data[var].dtypes == 'O']
cat = CatboostWrapper(CatBoostRegressor, SEED, cat_vars, cat_params, 'RMSE')
cat.train(train_x, train_y, val_x, val_y)

In [ ]:
lg_pred = lg.predict(test_x)
xg_pred = xg.predict(test_x)
cat_pred = cat.predict(test_x)

lg_result = np.sqrt(mean_squared_error(test_y, lg_pred))
xg_result = np.sqrt(mean_squared_error(test_y, xg_pred))
cat_result = np.sqrt(mean_squared_error(test_y, cat_pred))

print('RMSE for LightGBM Regressor: %.3f' % lg_result)
print('RMSE for XGB Regressor: %.3f' % xg_result)
print('RMSE for CatBoost Regressor: %.3f' % cat_result)